In [ ]:
#!pip install tensorflow
#!conda install tensorflow=${version} -y

In [ ]:
#Install these packages if not installed.  For tensorflow, the Pip method is preferred but I had trouble on Mac and
#   had to use conda.  Only need to do Pip or COnda and only if not installed.

#!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib """***Use this line if images are in google drive""""""
#!pip install opencv-python -y

#!pip install tensorflow
#!conda install tensorflow=${version} -y

In [1]:
#Import various libraries.  I dont beleive this will work in google colab as colab and openCV don't play well together 
#as of June 2023.  

import tensorflow as tf
import pandas as pd
import numpy as np
import os

from tensorflow.python.keras.utils.data_utils import Sequence

import cv2
from sklearn.model_selection  import train_test_split

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D,Dense,Flatten,Dropout
from tensorflow.keras.optimizers import Adam

import scipy.misc
import os


import pickle

import tensorflow as tf
keras = tf.compat.v1.keras #Had to go with a legacy version of Keras because of trouble with the datagenerator
Sequence = keras.utils.Sequence



from imutils import paths

#initial learning rate and dimensions of our images
INIT_LR = 1e-4
input_shape = (66, 200, 3)

2023-06-09 20:04:21.481406: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Parameters for datagen.py
params = {'dim': (66,200,3),
          'batch_size': 32,
          'shuffle': True}

In [3]:
#the path of the folder containing your training images

path_name="/Users/dianevaught/Documents/Jeff/ML_Projects/SelfDrivingSimulation/RoadImages0to57999/"

In [5]:
# This opens the key file and creates two lists, the angle (target) and the ids which is the file name without jpg

angle=[]
ids=[]

f= open("/Users/dianevaught/Documents/Jeff/ML_Projects/SelfDrivingSimulation/Nvidiadatakey - SelfDrivingCarNvidiadatakey0to57999.txt")                                 #read steering angles from disk and preprocess
data = f.read()
data = data.split()
for i in data:                                      #if the node end with ".jpg" ignore it. what we need is only angles
    if i[-1]=='g':
        i = i[:-4]
        ids.append(i)
    else:
        angle.append(float(i))

In [6]:


#train and test set ratio. We create two dictionaries for data batch generator. 
#partition consists of two lists that holds the train and validation image ids. labels is a dictionary that
#hold the angles.


ids = list(map(int, ids))
partition={'train':ids[:int(len(ids)*.8)],'validation':ids[-int(len(ids)*.2):]}
labels={}
for i in partition["train"]:
    labels[i]=float(angle[i])* scipy.pi / 180
for i in partition["validation"]:
    labels[i]=float(angle[i])* scipy.pi / 180

In [43]:
#This is the image processing.  It uses various techniques to improve training.  This offered a SIGNIFICANT
#improvement of the training data

def preProcessing(img):
  
  #crops image, the goal is to get the road and nothing else.  Will depend on camera placement
  img= img[120:235,:,:]

  #converts to an image that changes colors but helps the computer see lane lines
  img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)

  #blurs the image improving model performance
  img =cv2.GaussianBlur(img, (3,3),0)

  #resizes the image as this is what Nvidia trained on
  img=cv2.resize(img, (200, 66))

  #Normalizes image.  Changes color values from 0 to 255 to 0 to 1.  This is to ensure that no
  # large numbers become more important on the model
  img = img/255

  return img

In [44]:
# Data Generator that provides the images in batches.  This allows for faster training as a number of images can
#be process at once, but not so many that the memory fills and crashes the program. More robust computers can 
#imcrease the batch size to speed up training


class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(66,200,3), shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp =[self.list_IDs[k] for k in indexes]
        
        # Generate data
        X, y = self.__data_generation(list_IDs_temp)


        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' 
        # Initialization
        X = np.empty((self.batch_size, *self.dim))
        y = np.empty(self.batch_size)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):

            # Store sample.  This is the feature data stored in a NUMPY array
            image = cv2.imread("/Users/dianevaught/Documents/Jeff/ML_Projects/SelfDrivingSimulation/RoadImages0to57999/"+str(ID)+".jpg")     #read images from disk
            image=preProcessing(image)

            X[i,] = np.asarray(image)

            # Store class.  This is the target stored in a NUMPY array
            y[i] = np.asarray(float(self.labels[ID]))

        return X, y
        


In [45]:
# Generators  This tells the model what pictures to apply the DataGenerator to.
training_generator = DataGenerator(partition["train"], labels, **params)
validation_generator = DataGenerator(partition["validation"], labels, **params)

In [39]:
#our model is Nvidia Dave-2 which you can find here: https://arxiv.org/pdf/1604.07316.pdf
def defineModel():
    model = Sequential()

    # 5x5 Convolutional layers with stride of 2x2
    model.add(Conv2D(24, (5, 5), strides=(2, 2),activation='elu',input_shape=input_shape))
    model.add(Conv2D(36, (5, 5), strides=(2, 2),activation='elu'))
    model.add(Conv2D(48, (5, 5), strides=(2, 2),activation='elu'))
    
    # 3x3 Convolutional layers with stride of 1x1
    model.add(Conv2D(64, (3, 3),activation='elu'))
    model.add(Conv2D(64, (3, 3),activation='elu'))
    
    # Flatten before passing to the fully connected layers
    model.add(Flatten())
    # Three fully connected layers
    model.add(Dense(100,activation='elu'))
    model.add(Dropout(.2))
    model.add(Dense(50,activation='elu'))
    model.add(Dropout(.2))
    model.add(Dense(10,activation='elu'))
    model.add(Dropout(.2))
    
    # Output layer with linear activation 
    model.add(Dense(1,activation="linear"))
    
    return model

In [41]:
#This compiles the model before training
model=defineModel()
model.compile(optimizer="adam", loss="mse")

In [46]:
#train the model for 10 epochs


model.fit(training_generator,
                    epochs=10,   
                    validation_data=validation_generator, verbose=2)

Epoch 1/10
1450/1450 - 234s - loss: 0.1755 - val_loss: 0.1253 - 234s/epoch - 162ms/step
Epoch 2/10
1450/1450 - 234s - loss: 0.1332 - val_loss: 0.1057 - 234s/epoch - 161ms/step
Epoch 3/10
1450/1450 - 227s - loss: 0.1126 - val_loss: 0.1149 - 227s/epoch - 157ms/step
Epoch 4/10
1450/1450 - 221s - loss: 0.0974 - val_loss: 0.1888 - 221s/epoch - 152ms/step
Epoch 5/10
1450/1450 - 259s - loss: 0.0971 - val_loss: 0.2766 - 259s/epoch - 179ms/step
Epoch 6/10
1450/1450 - 234s - loss: 0.0772 - val_loss: 0.2038 - 234s/epoch - 161ms/step
Epoch 7/10
1450/1450 - 227s - loss: 0.0759 - val_loss: 0.2604 - 227s/epoch - 156ms/step
Epoch 8/10
1450/1450 - 232s - loss: 0.0746 - val_loss: 0.1543 - 232s/epoch - 160ms/step
Epoch 9/10
1450/1450 - 227s - loss: 0.0721 - val_loss: 0.1292 - 227s/epoch - 156ms/step
Epoch 10/10
1450/1450 - 231s - loss: 0.0715 - val_loss: 0.2529 - 231s/epoch - 160ms/step


In [47]:
# save the model to disk
filename = '/Users/dianevaught/Documents/Jeff/ML_Projects/SelfDrivingSimulation/models/finalized_model2.sav'
pickle.dump(model, open(filename, 'wb'))
 


INFO:tensorflow:Assets written to: ram://d3e960ad-ac72-4ae3-954e-8dac1dbe99e1/assets


INFO:tensorflow:Assets written to: ram://d3e960ad-ac72-4ae3-954e-8dac1dbe99e1/assets
